This notebook demonstrates the main features of the AACProcessors library. First, let's install the package:


In [1]:
!pip install aac-processors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.0 MB/s eta 0:00:00


## Download Example Files
Let's get some example AAC files to work with

In [2]:
!wget https://raw.githubusercontent.com/willwade/AACProcessors/main/examples/demofiles/SimpleTest.gridset

--2025-01-17 05:46:03--  https://raw.githubusercontent.com/willwade/AACProcessors/main/examples/demofiles/SimpleTest.gridset
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 366974 (358K) [application/octet-stream]
Saving to: ‘SimpleTest.gridset’

SimpleTest.gridset  100%[===================>] 358.37K  --.-KB/s    in 0.03s   

2025-01-17 05:46:03 (11.3 MB/s) - ‘SimpleTest.gridset’ saved [366974/366974]



1. View File Structure. Let's examine the structure of a Grid3 file:

In [3]:
import os

from aac_processors import GridsetProcessor, viewer

# Initialize processor
processor = GridsetProcessor()
gridset_file = "SimpleTest.gridset"
if os.path.exists(gridset_file):
    print(f"Viewing structure of {os.path.basename(gridset_file)}")
    tree = processor.load_into_tree(gridset_file)
    viewer.print_tree(tree)

Viewing structure of SimpleTest.gridset

=== AAC Board Structure ===

Root Page:
  📄 Start (4x4 grid)
    Row 0:
      [Empty] (0, 0)
      🔀 quick chat (0, 1)
        └─ Says: quick chat
        └─ Goes to: quick chat
      🔀 don't like (0, 2)
        └─ Says: don't like
        └─ Goes to: Don't like
      🔀 something different (0, 3)
        └─ Says: something different
        └─ Goes to: something different
    Row 1:
      [Empty] (1, 0)
      🔀 something's wrong (1, 1)
        └─ Says: something's wrong
        └─ Goes to: something's wrong
      🔀 I want (1, 2)
        └─ Says: I want
        └─ Goes to: I want
      🔀 Comment (1, 3)
        └─ Says: Comment
        └─ Goes to: Comment
    Row 2:
      [Empty] (2, 0)
      🔀 About me (2, 1)
        └─ Says: About me
        └─ Goes to: About me
      └─ Target Page:
          📄 About me (5x4 grid)
            Row 0:
              🗣️ [No Label] (0, 0)
              🗣️ Back (0, 1)
                └─ Says: Back
              🔀 Fam

Extract some texts..

In [4]:
texts = processor.extract_texts(gridset_file)
print(f"\nExtracted {len(texts)} texts from {os.path.basename(gridset_file)}")
print("Sample texts:", texts[:5])


Extracted 43 texts from SimpleTest.gridset
Sample texts: ['Mum', 'orange squash', 'J 2 O', 'Age', 'something different']


In [5]:
# Install necessary library if not already installed
try:
    from deep_translator import GoogleTranslator
except ImportError:
    !pip install deep-translator
    from deep_translator import GoogleTranslator

# Translate texts using deep-translator
translations = {}
translator = GoogleTranslator(source="auto", target="es")

for text in texts:
    if text and isinstance(text, str):  # Check if text is not empty and is a string
        try:
            translated = translator.translate(text)
            translations[text] = translated
            print(f"Translated '{text}' to '{translated}'")
        except Exception as e:
            print(f"Error translating '{text}': {e}")
            translations[text] = text  # Keep original text if translation fails

# Process and save the translated gridset file
translated_file = processor.process_texts(
    gridset_file, translations, "translated_SimpleTest.gridset"
)
print(f"Translated gridset saved to: {translated_file}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00
Translated 'Mum' to 'Mamá'
Translated 'orange squash' to 'Calabaza naranja'
Translated 'J 2 O' to 'J 2 O'
Translated 'Age' to 'Edad'
Translated 'something different' to 'algo diferente'
Translated 'Back to home page' to 'Volver a la página de inicio'
Translated 'Comment' to 'Comentario'
Translated 'I feel' to 'Siento'
Translated 'Name' to 'Nombre'
Translated 'happy.' to 'feliz.'
Translated 'Milk shake' to 'Batido'
Translated 'Family' to 'Familia'
Translated 'don't like' to 'No me gusta'
Translated 'I'm asking a question' to 'Estoy haciendo una pregunta'
Translated 'blackcurrant squash' to 'calabaza grosella negra'
Translated 'Younger Sister' to 'Hermana menor'
Translated 'tired.' to 'cansado.'
Translated 'About me' to 'Acerca de mí'
Translated 'sad.' to 'triste.'
Translated 'excited.' to 'entusiasmado.'
Translated 'Home' to 'Hogar'
Translated 'Back' to 'Atrás'
Translated 'Dad' to 'Papá'
Translated 'Birthday' 

In [6]:
from aac_processors.cli import convert_format

# After creating translated_SimpleTest.gridset
output_obz = convert_format(
    input_file="translated_SimpleTest.gridset",
    output_format="coughdrop",
    output_path="translated_grid.obz",
)

if output_obz:
    print(f"Successfully converted to OBZ: {output_obz}")
else:
    print("Conversion failed")

Loading translated_SimpleTest.gridset
Loaded tree with 6 pages
Converting to coughdrop format at translated_grid.obz
Successfully converted to OBZ: translated_grid.obz


In [7]:
import json
import subprocess

from IPython.display import Image, display

if output_obz:
    print(f"Successfully converted to OBZ: {output_obz}")

    try:
        # Upload the file to obfviewer
        upload_command = [
            "curl",
            "-X",
            "POST",
            "-H",
            "Content-Type: application/zip",
            "--data-binary",
            f"@{output_obz}",
            "https://obfviewer.acecentre.net/api/upload",
        ]

        result = subprocess.run(upload_command, capture_output=True, text=True)

        if result.returncode == 0:
            response_json = json.loads(result.stdout)

            if "renderedImage" in response_json:
                image_data = response_json["renderedImage"]
                image_data = image_data.split(",", 1)[1]  # Remove data URL prefix

                with open("board.png", "wb") as f:
                    f.write(
                        bytes.fromhex(image_data.replace("\n", ""))
                    )  # added for fix base64 decoding
                display(Image("board.png"))

            else:
                print("Error: 'renderedImage' not found in the response.")
                print("Response:", json.dumps(response_json, indent=2))

        else:
            print("Error uploading file:")
            print(result.stderr)

    except Exception as e:
        print(f"An error occurred during the upload or image extraction process: {e}")

else:
    print("Conversion failed")

Successfully converted to OBZ: translated_grid.obz
Error: 'renderedImage' not found in the response.
Response: {
  "error": "Failed to process OBZ file",
  "details": "a.images is not iterable"
}
